# DRQN 구현


In [2]:
import numpy as np
import random
import tensorflow as tf
import matplotlib.pyplot as plt
import scipy.misc
import os
import csv
import itertools
import tensorflow.contrib.slim as slim
%matplotlib inline

from helper import *

In [1]:
from gridworld import gameEnv

env = gameEnv(partial=True, size=9)

In [ ]:
class Qnetwork():
    def __init__(self,h_size,rnn_cell,myScope):
        # 네트워크는 게임으로부터 하나의 프레임을 받아 이를 배열로 만든다(flattening).
        # 그 다음 배열의 크기를 재조절하고 4개의 합성곱 계층을 거처 처리한다.
        self.scalarInput = tf.placeholder(shape=[None,21168],dtype=tf.loat32)
        self.imageIn = tf.reshape(self.scalarInput, shape=[-1,84,84,3])
        self.conf1 = slim.convolution2d(inputs=self.imageIn,num_outputs=32,\
                                       kernel_size=[8,8],stride=[4,4],padding='VALID',\
                                       biases_initalizer=None,scope=myScope+'_conv1')
        self.conf2 = slim.convolution2d(inputs=self.imageIn,num_outputs=32,\
                                       kernel_size=[4,4],stride=[2,2],padding='VALID',\
                                       biases_initalizer=None,scope=myScope+'_conv2')
        self.conf3 = slim.convolution2d(inputs=self.imageIn,num_outputs=32,\
                                       kernel_size=[3,3],stride=[1,1],padding='VALID',\
                                       biases_initalizer=None,scope=myScope+'_conv3')
        self.conf4 = slim.convolution2d(inputs=self.imageIn,num_outputs=32,\
                                       kernel_size=[7,7],stride=[1,1],padding='VALID',\
                                       biases_initalizer=None,scope=myScope+'_conv4')
        
        self.trainLength = tf.placeholder(dtype=tf.int32)
        # 마지막 합성곱 계층에서 출력값을 취한 후 이를 순환 계층에 보낸다.
        # 입력값은 RNN 처리를 위해 [batch x trace x units]으로 크기를 바꿔야 하며
        # 상위 레벨로 전달될 때 [batch x units] 형태로 리턴해야 한다.
        self.batch_size = tf.placeholder(dtype=tf.int32)
        self.confFlat = tf.reshape(slim.flatten(self.conv4), [self.batch_size,self.trainLength,h_size])
        tt